<a href="https://colab.research.google.com/github/areias/slm-finetunig/blob/main/finetune_mistral.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Fine-tuning Mistral-7b for relationship extraction

See how mistral-7b performs at relationship extraction task before and after fine-tuning on 1000 examples.

based on

[Fine-tune large models using 🤗 peft adapters, transformers & bitsandbytes](https://colab.research.google.com/drive/1jCkpikz0J2o20FBQmYmAGdiKmJGOMo-o?usp=sharing#scrollTo=WE5GJ6s7y0Xo)


[transformers meets bitsandbytes for democratzing Large Language Models (LLMs) through 4bit quantization](https://colab.research.google.com/drive/1VoYNfYDKcKRQRor98Zbf2-9VQTtGJ24k#scrollTo=jq0nX33BmfaC)

[Fine-tuning Mistral on your own data](https://github.com/brevdev/notebooks/blob/main/mistral-finetune-own-data.ipynb)

[Fine-tuning Large Language Models](https://learn.deeplearning.ai/finetuning-large-language-models/lesson/1/introduction)


[Ollama: Running GGUF Models from Hugging Face](https://www.markhneedham.com/blog/2023/10/18/ollama-hugging-face-gguf-models/)





## Load our fine-tuning data

In [15]:
! pip install -q peft
! pip install -q git+https://github.com/huggingface/accelerate.git
! pip install -q bitsandbytes
! pip install -q transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 6.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.0 MB/s eta 0:00:00


In [19]:
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [35]:
!ls

drive  sample_data


In [1]:
import sys
print(sys.path)

['/content', '/env/python', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.10/dist-packages/IPython/extensions', '/root/.ipython']


In [2]:
import sys
sys.path.append('drive/My Drive/mistral-finetune/')

In [3]:
import pandas as pd
df = pd.read_csv("drive/MyDrive/mistral-finetune/data/nyt10m_finetuning-balanced.csv")

In [4]:
len(df['text'].unique())

974

In [5]:
len(df)

1452

In [6]:
df.head()

,text,h_name,t_name,relation
0,There were also performers who were born in Lo...,Johnny Rivers,Louisiana,/people/person/place_of_birth
1,"Indeed , Mr. Chávez said Sunday that he wished...",Evo Morales,Bolivia,/people/person/nationality
2,"Indeed , Mr. Chávez said Sunday that he wished...",Evo Morales,Bolivia,/people/person/place_lived
3,Analysts contrast Mr. Chávez 's success at con...,Evo Morales,Bolivia,/people/person/nationality
4,"Ecuador 's new president , Rafael Correa , joi...",Evo Morales,Bolivia,/people/person/nationality


In [7]:
df.loc[df['relation'].isna(),'relation']="None"
#df = df.loc[~df['relation'].isna()] #remove Nones


In [8]:
sentence=df['text'].unique()[31]
sentence

"Among the mourners were Queen Sofía of Spain and Bernadette Chirac , the wife of President Jacques Chirac of France , President Ilham Aliyev of Azerbaijan and young cellists , bearing witness to Mr. Rostropovich 's reputation as a man of the world whose appeal cut across class lines ."

In [9]:
df.relation.value_counts() / len(df)

/people/person/place_lived                    0.176997
/location/located_in                          0.153581
/business/person/company                      0.125344
/location/administrative_division/country     0.082645
/people/person/nationality                    0.081956
None                                          0.081956
/business/location                            0.079890
/location/country/administrative_divisions    0.066804
/people/person/place_of_birth                 0.058540
/business/company/advisors                    0.048898
/people/deceasedperson/place_of_death         0.043388
Name: relation, dtype: float64

In [10]:
df[df['text']==sentence]

,text,h_name,t_name,relation
45,Among the mourners were Queen Sofía of Spain a...,Jacques Chirac,France,/people/person/nationality
46,Among the mourners were Queen Sofía of Spain a...,Jacques Chirac,France,/people/person/place_lived
798,Among the mourners were Queen Sofía of Spain a...,Bernadette Chirac,France,/people/person/nationality
799,Among the mourners were Queen Sofía of Spain a...,Bernadette Chirac,France,/people/person/place_lived


In [11]:
len(df)

1452

In [12]:
df.text.nunique()

974

In [13]:
! ls drive/MyDrive/mistral-finetune/

data					 mistral-finetuning-data-r128alpha128batch2.json
helper_functions.py			 mistral-finetuning-data-r128alpha64batch8.json
mistral-finetuning-data2.json		 mistral-finetuning-data-r128alphasqrt128batch8.json
mistral-finetuning-data3.json		 mistral-finetuning-data-r32alpha16.json
mistral-finetuning-data-balanced.json	 mistral-finetuning-data-r8alpha16.json
mistral-finetuning-data.json		 __pycache__
mistral-finetuning-data-loweralpha.json  runs
mistral-finetuning-data-nonegative.json


In [14]:
import numpy as np
from helper_functions import preprocess

In [15]:
# make sure works for example with no relations
data = preprocess(df)
data[400]

{'input': 'Yesterday , Mr. Sulzberger received support when Donald E. Graham , the chairman and chief executive of the Washington Post Company , urged investors to support the Times Company .',
 'output': "('Washington Post Company', '/business/company/advisors', 'Donald E. Graham'), ('Donald E. Graham', '/business/person/company', 'Washington Post Company')"}

In [16]:
# and exapmle with multiple relations
data[230]

{'input': 'The Real Estate Market Though prices have risen sharply in recent years , real estate agents and developers still talk up Port St. Lucie as a Florida bargain .',
 'output': "('St. Lucie', '/location/located_in', 'Florida')"}

In [17]:
# as well as your single relation
data[:4]

[{'input': "There were also performers who were born in Louisiana , including Lucinda Williams , Jerry Lee Lewis and Johnny Rivers , whose '' Secret Agent Man '' had a touch of bayou-country swamp-pop .",
  'output': "('Johnny Rivers', '/people/person/place_of_birth', 'Louisiana'), ('Jerry Lee Lewis', '/people/person/place_of_birth', 'Louisiana')"},
 {'input': "Indeed , Mr. Chávez said Sunday that he wished Mr. Correa the '' best of luck '' from Venezuela , where he appeared on television with President Evo Morales of Bolivia to commemorate the opening of a milk processing plant built in western Venezuela with financing from Iran .",
  'output': "('Evo Morales', '/people/person/nationality', 'Bolivia'), ('Evo Morales', '/people/person/place_lived', 'Bolivia')"},
 {'input': "Analysts contrast Mr. Chávez 's success at consolidating power in Venezuela with the troubles faced by President Evo Morales in Bolivia , where a constitution-writing assembly has been stalled by fractious delegates

In [18]:
df.relation.value_counts(dropna=False)

/people/person/place_lived                    257
/location/located_in                          223
/business/person/company                      182
/location/administrative_division/country     120
/people/person/nationality                    119
None                                          119
/business/location                            116
/location/country/administrative_divisions     97
/people/person/place_of_birth                  85
/business/company/advisors                     71
/people/deceasedperson/place_of_death          63
Name: relation, dtype: int64

In [19]:
relations = np.sort(list(df.relation.unique()))
relations = list(relations[:len(relations)-1])
relations


['/business/company/advisors',
 '/business/location',
 '/business/person/company',
 '/location/administrative_division/country',
 '/location/country/administrative_divisions',
 '/location/located_in',
 '/people/deceasedperson/place_of_death',
 '/people/person/nationality',
 '/people/person/place_lived',
 '/people/person/place_of_birth']

In [20]:
from helper_functions import formatting_func, eval_formatting_func

In [21]:
print(formatting_func(data[0]))

[INST] You are an NLP expert tasked with entity and relation extraction. Here is a list of potential relation types: ['/business/company/advisors', '/business/location', '/business/person/company', '/location/administrative_division/country', '/location/country/administrative_divisions', '/location/located_in', '/people/deceasedperson/place_of_death', '/people/person/nationality', '/people/person/place_lived', '/people/person/place_of_birth'] 
What relations in the given list might be included in the given sentence? Make sure to use only relations from the list provided!
The given sentence is: 'There were also performers who were born in Louisiana , including Lucinda Williams , Jerry Lee Lewis and Johnny Rivers , whose '' Secret Agent Man '' had a touch of bayou-country swamp-pop .'
If none present, answer: None.
Respond with relationship triples, e.g. (head entity 1, relation 1, tail entity 1), (head entity 2, relation 2, tail entity 2), (...) 
Take care, your answer is only valid if 

In [22]:
print(eval_formatting_func(data[0]))

[INST] You are an NLP expert tasked with entity and relation extraction. Here is a list of potential relation types: ['/business/company/advisors', '/business/location', '/business/person/company', '/location/administrative_division/country', '/location/country/administrative_divisions', '/location/located_in', '/people/deceasedperson/place_of_death', '/people/person/nationality', '/people/person/place_lived', '/people/person/place_of_birth'] 
What relations in the given list might be included in the given sentence? Make sure to use only relations from the list provided!
The given sentence is: 'There were also performers who were born in Louisiana , including Lucinda Williams , Jerry Lee Lewis and Johnny Rivers , whose '' Secret Agent Man '' had a touch of bayou-country swamp-pop .'
If none present, answer: None.
Respond with relationship triples, e.g. (head entity 1, relation 1, tail entity 1), (head entity 2, relation 2, tail entity 2), (...) 
Take care, your answer is only valid if 

## Load a quantized Mistral:instruct model

https://huggingface.co/docs/peft/main/en/developer_guides/quantization


...since PEFT methods only add extra trainable parameters, this allows you to train a quantized model with a PEFT adapter on top! Combining quantization with PEFT can be a good strategy for training even the largest models on a single GPU. For example, QLoRA is a method that quantizes a model to 4-bits and then trains it with LoRA.


bitsandbytes is a quantization library with a Transformers integration. With this integration, you can quantize a model to 8 or 4-bits and enable many other options by configuring the BitsAndBytesConfig class. For example, you can:

In [23]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

 https://huggingface.co/docs/peft/main/en/developer_guides/quantization

...For example, you can:

* set load_in_4bit=True to quantize the model to 4-bits when you load it
* set bnb_4bit_quant_type="nf4" to use a special 4-bit data type for weights initialized from a normal distribution
* set bnb_4bit_use_double_quant=True to use a nested quantization scheme to quantize the already quantized weights
* set bnb_4bit_compute_dtype=torch.bfloat16 to use bfloat16 for faster computation

In [24]:
model_id = "mistralai/Mistral-7B-Instruct-v0.2"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)


Pass the config to the from_pretrained method.

In [25]:
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

## Load tokenizer

https://huggingface.co/docs/transformers/en/llm_tutorial#wrong-padding-side

LLMs are decoder-only architectures, meaning they continue to iterate on your input prompt. If your inputs do not have the same length, they need to be padded. Since LLMs are not trained to continue from pad tokens, your input needs to be left-padded. Make sure you also don’t forget to pass the attention mask to generate!

tokenizer.pad_token = tokenizer.eos_token  # Most LLMs don't have a pad token by default


In [26]:
tokenizer = AutoTokenizer.from_pretrained(model_id,
                                        padding_side="left") # padding on left from brewdev notebook
tokenizer.pad_token = tokenizer.eos_token


tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]



Now let's tokenize again with padding and truncation, and set up the tokenize function to make labels and input_ids the same. This is basically what self-supervised fine-tuning is.


In [27]:
max_length = 512 # This was an appropriate max length for my dataset

def generate_and_tokenize_prompt(prompt):
    result = tokenizer(
        formatting_func(prompt),
        truncation=True,
        max_length=max_length,
        padding="max_length",
    )
    result["labels"] = result["input_ids"].copy()
    return result

## Evaluate model out of the box



In [28]:
from helper_functions import get_relations, get_tuples, get_scores

In [ ]:
"""eval_prompt = formatting_func(data[230])
print(eval_prompt)"""

'eval_prompt = formatting_func(data[230])\nprint(eval_prompt)'

In [29]:
# Init an eval tokenizer that doesn't add padding or eos token
eval_tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    add_bos_token=True,
)


https://huggingface.co/docs/transformers/en/generation_strategies



In [ ]:
"""model_input = eval_tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    response  = eval_tokenizer.decode(model.generate(**model_input,
                    max_new_tokens=256, repetition_penalty=1.15,
                    pad_token_id=eval_tokenizer.eos_token_id)[0], skip_special_tokens=True)
    response = response.replace(eval_prompt, "")
    print(response)"""

'model_input = eval_tokenizer(eval_prompt, return_tensors="pt").to("cuda")\n\nmodel.eval()\nwith torch.no_grad():\n    response  = eval_tokenizer.decode(model.generate(**model_input,\n                    max_new_tokens=256, repetition_penalty=1.15,\n                    pad_token_id=eval_tokenizer.eos_token_id)[0], skip_special_tokens=True)\n    response = response.replace(eval_prompt, "")\n    print(response)'

In [ ]:
"""response"""

'response'

In [ ]:
"""predicted_relations = get_tuples(response.replace(eval_prompt, ""))
predicted_relations"""

'predicted_relations = get_tuples(response.replace(eval_prompt, ""))\npredicted_relations'

In [ ]:
#predicted_relations.append(('Evo Morales', '/people/person/nationality', 'Bolivia'))

In [ ]:
#predicted_relations

In [ ]:
"""import helper_functions
from importlib import reload
reload(helper_functions)"""

'import helper_functions\nfrom importlib import reload\nreload(helper_functions)'

In [ ]:
"""expected_relations = get_relations(data[230])
expected_relations"""

'expected_relations = get_relations(data[230])\nexpected_relations'

In [ ]:
"""get_scores(expected_relations, ["None"])"""

'get_scores(expected_relations, ["None"])'

In [ ]:
"""import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding"""

'import locale\ndef getpreferredencoding(do_setlocale = True):\n    return "UTF-8"\nlocale.getpreferredencoding = getpreferredencoding'

In [ ]:
"""pip install pyarrow==11.0.0"""

'pip install pyarrow==11.0.0'

In [30]:
from sklearn.model_selection import train_test_split
from datasets import Dataset

# Convert list of dictionaries to DataFrame
df = pd.DataFrame(data)

# Split DataFrame into train and test
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Convert back to datasets
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)



In [31]:
# Optionally, you can remove the pandas index column
train_dataset = train_dataset.remove_columns('__index_level_0__')
test_dataset = test_dataset.remove_columns('__index_level_0__')



In [32]:
train_dataset

Dataset({
    features: ['input', 'output'],
    num_rows: 779
})

In [33]:
tokenized_train_dataset = train_dataset.map(generate_and_tokenize_prompt)
tokenized_test_dataset = test_dataset.map(generate_and_tokenize_prompt)

Map:   0%|          | 0/779 [00:00<?, ? examples/s]

Map:   0%|          | 0/195 [00:00<?, ? examples/s]

In [34]:
tokenizer.decode(tokenized_train_dataset[1]['input_ids'])

"</s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s><s> [INST] You are an NLP expert tasked with entity and relation extraction. Here is a list of potential relation types: ['/business/company/advisors', '/business/location', '/business/person/company', '/location/administrative_division/country', '/location/country/administrative_divisions', '/location/located_in', '/peop

In [ ]:
"""len(tokenized_train_dataset[1]['input_ids'])"""

"len(tokenized_train_dataset[1]['input_ids'])"

In [ ]:
"""print(tokenized_train_dataset[1]['input_ids'])"""

"print(tokenized_train_dataset[1]['input_ids'])"

In [ ]:
"""import matplotlib.pyplot as plt

def plot_data_lengths(tokenized_train_dataset, tokenized_test_dataset):
    lengths = [len(x['input_ids']) for x in tokenized_train_dataset]
    lengths += [len(x['input_ids']) for x in tokenized_test_dataset]
    print(len(lengths))

    # Plotting the histogram
    plt.figure(figsize=(10, 6))
    plt.hist(lengths, bins=20, alpha=0.7, color='blue')
    plt.xlabel('Length of input_ids')
    plt.ylabel('Frequency')
    plt.title('Distribution of Lengths of input_ids')
    plt.show()

plot_data_lengths(tokenized_train_dataset, tokenized_test_dataset)"""

"import matplotlib.pyplot as plt\n\ndef plot_data_lengths(tokenized_train_dataset, tokenized_test_dataset):\n    lengths = [len(x['input_ids']) for x in tokenized_train_dataset]\n    lengths += [len(x['input_ids']) for x in tokenized_test_dataset]\n    print(len(lengths))\n\n    # Plotting the histogram\n    plt.figure(figsize=(10, 6))\n    plt.hist(lengths, bins=20, alpha=0.7, color='blue')\n    plt.xlabel('Length of input_ids')\n    plt.ylabel('Frequency')\n    plt.title('Distribution of Lengths of input_ids')\n    plt.show()\n\nplot_data_lengths(tokenized_train_dataset, tokenized_test_dataset)"

## Set up LoRA

https://www.datacamp.com/tutorial/mastering-low-rank-adaptation-lora-enhancing-large-language-models-for-efficient-adaptation

LoRA is built on the understanding that large models inherently possess a low-dimensional structure. By leveraging low-rank matrices, LoRA adapts these models effectively. This method focuses on the core concept that significant model changes can be represented with fewer parameters, thus making the adaptation process more efficient.

First, we decompose the large weight matrices into smaller matrices using the lower-rank matrix technique, as explained above. This drastically reduces the number of trainable parameters. For a model like GPT-3, trainable parameters are reduced by 10000 times. This means instead of training 175 billion parameters, if you apply LoRA, you only have 17.5 million trainable parameters.

We do not change any parameters for a pre-trained model. Instead, only train lower-rank matrices, which happen relatively very quickly because of fewer parameters.

The weights are additive. This means for inference, we just add the weights of lower-rank matrices to pre-trained weights without any additional latency. The lower-rank matrices are very small in size as well so it is very easy to load and unload them for different tasks and different users

LoRA does not introduce any additional latency during inference. While low-rank matrices are used during training, they are merged with the original parameters for inference, ensuring no slowdown





Next, you should call the prepare_model_for_kbit_training() function to preprocess the quantized model for traininng.

In [35]:
from peft import prepare_model_for_kbit_training

#model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

https://huggingface.co/docs/peft/main/en/developer_guides/lora#qlora-style-training

However, in QLoRA, it was found that adding trainable weights to all the linear layers of a transformer model is beneficial to match full-finetuning performance. Since the list of modules to add will vary depending on the architecture, we provided a convenient shorthand : simple specify target_modules='all-linear' and let 🤗PEFT handle the rest:

https://huggingface.co/blog/4bit-transformers-bitsandbytes


## opinions on LoRA params

https://medium.com/@fartypantsham/what-rank-r-and-alpha-to-use-in-lora-in-llm-1b4f025fd133

Generally ranks bellow 32 are more like a smudges on a glass with a dirty rag. Very low values like 8 or 4 are basically just telling the model a style of how you want the output to be formatted to but not care about the details as there isn’t enough parameters to distinguish much more.

Alpha is a scaling parameter.

alpha = rank is scaling weights at 1.0

What you train in LORA weights will be then merged with the main weights of model at x 1.0

Previously people were suggesting alpha = (2 x rank), which is like yelling at your model really loud — all in order to make the newly learned weights “louder” than the model’s own. That requires a really good and large dataset, otherwise you are just amplifying nonsense.

The model knows how to speak well already, while your dataset is too small to teach (or scream at) the model any language fundamentals. Increasing alpha amplifies everything, not just the stuff you wish the model learns from it.

I would suggest rank = alpha, most of the time as your base — because it is very easily to attenuate the LORA data after the training is done if it appears to be too “loud”, overtaking the entire model.



https://www.anyscale.com/blog/fine-tuning-llms-lora-or-full-parameter-an-in-depth-analysis-with-llama-2

* Rank: 8 Choosing a higher rank for our decomposition matrices would counteract LoRA's efficiency gains. Our preliminary tests suggested minimal performance boosts when increasing the rank to, for instance, 16. As a result, we settled on a rank of 8 to maintain smaller checkpoint sizes and to avoid artificially inflating our checkpoint files.
* Alpha: 16 Alpha scales the learned weights. Existing literature, including the original LoRA paper, generally advises fixing Alpha—often at 16—rather than treating it as a tunable hyperparameter.
* Base learning rate: 1e-4
A learning rate of 1e-4 has become the standard when fine-tuning LLMs with LoRA. Although we occasionally encountered training loss instabilities, reducing the learning rate to lower values like 3e-5 proved effective in stabilizing the process—more on this will follow.

In [ ]:
128*2

256

In [ ]:
import math
math.sqrt(256)

16.0

In [36]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules="all-linear",
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)


In [37]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )



Let's print the model to examine its layers, as we will apply QLoRA to all the linear layers of the model. Those layers are q_proj, k_proj, v_proj, o_proj, gate_proj, up_proj, down_proj, and lm_head.


In [38]:
model = get_peft_model(model, config)

https://github.com/huggingface/transformers/blob/main/src/transformers/models/mistral/modeling_mistral.py

In [39]:
print_trainable_parameters(model)

trainable params: 41943040 || all params: 3794014208 || trainable%: 1.1055056122762943


In [40]:
print(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear4bit(
                (base_layer): Lin

https://huggingface.co/docs/transformers/peft

In [41]:
!pip install -q wandb -U

import wandb, os
wandb.login()

wandb_project = "mistral-finetune"
if len(wandb_project) > 0:
    os.environ["WANDB_PROJECT"] = wandb_project

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.5/258.5 kB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.0 MB/s eta 0:00:00


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
#mkdir drive/MyDrive/mistral-finetune/runs

In [ ]:
#mkdir drive/MyDrive/mistral-finetune/runs/logs

In [42]:

project = "re-finetune"
base_model_name = "mistral7b"
run_name = base_model_name + "-" + project  # rank 256 alpha 256 scale factor 1 and batch size 2
output_dir = "drive/MyDrive/mistral-finetune/runs/" + run_name


https://huggingface.co/docs/peft/main/en/conceptual_guides/lora#common-lora-parameters-in-peft



In [43]:
output_dir

'drive/MyDrive/mistral-finetune/runs/mistral7b-re-finetune'

bf16 gives error ValueError: Your setup doesn't support bf16/gpu. You need torch>=1.10, using Ampere GPU with cuda>=11.0

https://ai.google.dev/gemma/docs/lora_tuning#lora_fine-tuning

The LoRA rank determines the dimensionality of the trainable matrices that are added to the original weights of the LLM. It controls the expressiveness and precision of the fine-tuning adjustments.

A higher rank means more detailed changes are possible, but also means more trainable parameters. A lower rank means less computational overhead, but potentially less precise adaptation.

This tutorial uses a LoRA rank of 4. In practice, begin with a relatively small rank (such as 4, 8, 16). This is computationally efficient for experimentation. Train your model with this rank and evaluate the performance improvement on your task. Gradually increase the rank in subsequent trials and see if that further boosts performance.

https://huggingface.co/transformers/v3.0.2/main_classes/trainer.html

*  train_batch_size, per_device_train_batch_size (int, optional, defaults to 8) – The batch size per PU/TPU core/CPU for training.
* learning_rate (float, optional, defaults to 5e-5) – The initial learning rate for Adam.
* warmup_steps (int, optional, defaults to 0) – Number of steps used for a linear warmup from 0 to learning_rate.




In [44]:
from datetime import datetime
import transformers

trainer = transformers.Trainer(
    model=model,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
    args=transformers.TrainingArguments(
        output_dir=output_dir,
        warmup_steps=1,
        per_device_train_batch_size=2,
        gradient_accumulation_steps=1,
        gradient_checkpointing=True,
        max_steps=500,
        learning_rate=2.5e-5, # Want a small lr for finetuning
        fp16=True,
        logging_steps=25,
        optim="paged_adamw_8bit",
        logging_dir="drive/MyDrive/mistral-finetune/runs/logs",        # Directory for storing logs
        save_strategy="steps",       # Save the model checkpoint every logging step
        save_steps=25,                # Save checkpoints every 50 steps
        evaluation_strategy="steps", # Evaluate the model every logging step
        eval_steps=25,               # Evaluate and save checkpoints every 50 steps
        do_eval=True,                # Perform evaluation at the end of training
        report_to="wandb",           # Comment this out if you don't want to use weights & baises
        run_name=f"{run_name}-{datetime.now().strftime('%Y-%m-%d-%H-%M')}"          # Name of the W&B run (optional)
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()



/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


wandb: Currently logged in as: aareias. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss
25,1.309500,0.630795
50,0.559600,0.520468
75,0.508800,0.447116
100,0.455200,0.402771
125,0.384900,0.389356
150,0.382500,0.380317
175,0.404300,0.373686
200,0.385600,0.364334
225,0.375900,0.358685
250,0.360900,0.352990


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.

KeyboardInterrupt: 

https://magazine.sebastianraschka.com/p/practical-tips-for-finetuning-llms

batch size recommendations https://cloud.google.com/vertex-ai/docs/model-garden/lora-qlora

restart session!

## Evaluating

In [1]:
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')


Mounted at /content/drive


In [1]:
import sys
sys.path.append('drive/My Drive/mistral-finetune/')

In [2]:
from helper_functions import preprocess, eval_formatting_func

In [3]:
# load test data
import pandas as pd
df = pd.read_csv("./drive/MyDrive/mistral-finetune/data/nyt10m_test.csv")


In [4]:
df.loc[df['relation'].isna(),'relation']="None"

In [5]:
df.relation.value_counts()/len(df)

/people/person/place_lived                    0.229008
/location/located_in                          0.213740
None                                          0.190840
/business/person/company                      0.114504
/location/country/administrative_divisions    0.061069
/people/person/nationality                    0.061069
/location/administrative_division/country     0.053435
/business/location                            0.038168
/business/company/advisors                    0.015267
/people/deceasedperson/place_of_death         0.015267
/people/person/place_of_birth                 0.007634
Name: relation, dtype: float64

In [6]:
data = preprocess(df)

In [7]:
len(data)

100

In [8]:
data[2]

{'input': 'A1 OBITUARIES A15 Mstislav Rostropovich A cellist and conductor who was renowned as one of the great instrumentalists of the 20th century and an outspoken champion of artistic freedom in the Soviet Union during the last decades of the cold war , he was 80 and lived in Paris .',
 'output': "('Mstislav Rostropovich', '/people/person/place_lived', 'Paris')"}

In [9]:
import numpy as np
relations = np.sort(list(df.relation.unique()))
relations = list(relations[:len(relations)-1])
relations

['/business/company/advisors',
 '/business/location',
 '/business/person/company',
 '/location/administrative_division/country',
 '/location/country/administrative_divisions',
 '/location/located_in',
 '/people/deceasedperson/place_of_death',
 '/people/person/nationality',
 '/people/person/place_lived',
 '/people/person/place_of_birth']

In [10]:
""" double checking that fits training data same categories
['/business/company/advisors',
 '/business/location',
 '/business/person/company',
 '/location/administrative_division/country',
 '/location/country/administrative_divisions',
 '/location/located_in',
 '/people/deceasedperson/place_of_death',
 '/people/person/nationality',
 '/people/person/place_lived',
 '/people/person/place_of_birth']"""

" double checking that fits training data same categories\n['/business/company/advisors',\n '/business/location',\n '/business/person/company',\n '/location/administrative_division/country',\n '/location/country/administrative_divisions',\n '/location/located_in',\n '/people/deceasedperson/place_of_death',\n '/people/person/nationality',\n '/people/person/place_lived',\n '/people/person/place_of_birth']"

In [11]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

In [12]:
model_id = "mistralai/Mistral-7B-Instruct-v0.2"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [13]:
# Init an eval tokenizer that doesn't add padding or eos token
eval_tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    add_bos_token=True,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [ ]:
"""pip install accelerate"""

In [14]:
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [16]:
eval_prompt = eval_formatting_func(data[2])


In [17]:
eval_prompt

"[INST] You are an NLP expert tasked with entity and relation extraction. Here is a list of potential relation types: ['/business/company/advisors', '/business/location', '/business/person/company', '/location/administrative_division/country', '/location/country/administrative_divisions', '/location/located_in', '/people/deceasedperson/place_of_death', '/people/person/nationality', '/people/person/place_lived', '/people/person/place_of_birth'] \nWhat relations in the given list might be included in the given sentence? Make sure to use only relations from the list provided!\nThe given sentence is: 'A1 OBITUARIES A15 Mstislav Rostropovich A cellist and conductor who was renowned as one of the great instrumentalists of the 20th century and an outspoken champion of artistic freedom in the Soviet Union during the last decades of the cold war , he was 80 and lived in Paris .'\nIf none present, answer: None.\nRespond with relationship triples, e.g. (head entity 1, relation 1, tail entity 1), 

In [ ]:
model_input = eval_tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    response  = eval_tokenizer.decode(model.generate(**model_input, max_new_tokens=256, repetition_penalty=1.15)[0], skip_special_tokens=True)
    response = response.replace(eval_prompt, "")
    print(response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 Based on the given sentence, the following relation triple can be extracted:
(Mstislav Rostropovich, /people/person/place_lived, Paris)


In [ ]:
predicted_relations = get_tuples(response)
predicted_relations

[('Mstislav Rostropovich', '/people/person/place_lived', 'Paris')]

In [ ]:
expected_relations = get_relations(data[10])
expected_relations

['None']

In [ ]:
expected_set = set(expected_relations)
expected_set

{'None'}

In [ ]:
precision, recall,f1 = get_scores(expected_relations, predicted_relations)

In [ ]:
precision,recall,f1

(1.0, 1.0, 1.0)

In [ ]:
data[0].update({'base_model' : {'predicted_relations':predicted_relations,
                                'precision': precision,
                                'recall': recall,
                                'f1': f1}})

In [ ]:
data[1:3]

[{'input': "9:30 P.M. (13) THE WILD PARROTS OF TELEGRAPH HILL -- '' Independent Lens '' follows Mark Bittner , a homeless street musician in San Francisco , as he literally goes to the birds -- and finds meaning in his life -- by taking up with a flock of wild red and green parrots .",
  'output': "('Mark Bittner', '/people/person/place_lived', 'San Francisco')"},
 {'input': 'A1 OBITUARIES A15 Mstislav Rostropovich A cellist and conductor who was renowned as one of the great instrumentalists of the 20th century and an outspoken champion of artistic freedom in the Soviet Union during the last decades of the cold war , he was 80 and lived in Paris .',
  'output': "('Mstislav Rostropovich', '/people/person/place_lived', 'Paris')"}]

In [ ]:
list(range(1,3))

[1, 2]

In [ ]:
from tqdm import tqdm

progress_bar = tqdm(total=len(data), desc='Progress')

for idx, example in zip(list(range(len(data))), data):

    eval_prompt = formatting_func(example)

    model_input = eval_tokenizer(eval_prompt, return_tensors="pt").to("cuda")

    model.eval()
    with torch.no_grad():
        response  = eval_tokenizer.decode(model.generate(**model_input, max_new_tokens=256,
                                                         repetition_penalty=1.15,
                                                         pad_token_id=eval_tokenizer.eos_token_id)[0],
                                          skip_special_tokens=True)
        response = response.replace(eval_prompt, "")

    predicted_relations = get_tuples(response)

    expected_relations = get_relations(example)

    precision, recall,f1 = get_scores(expected_relations, predicted_relations)

    data[idx].update({'base_model':
                        {'response': response,
                         'predicted_relations':predicted_relations,
                        'precision': precision,
                        'recall': recall,
                        'f1': f1}
                    })
    progress_bar.update(1)

progress_bar.close()

Progress: 100%|██████████| 100/100 [22:17<00:00, 13.38s/it]


In [ ]:
data[:4]

[{'input': '-LRB- AP -RRB- WEST VIRGINIA 67 , VILLANOVA 56 -- Frank Young scored a career-high 25 points , including eight 3-pointers , for No. 25 West Virginia -LRB- 12-1 , 2-0 -RRB- in a Big East game .',
  'output': "('Frank Young', '/people/person/place_lived', 'West Virginia')",
  'base_model': {'response': " Based on the given sentence, there do not seem to be any explicit relations mentioned that can be directly extracted using the provided list. The sentence primarily describes an event (a basketball game) involving entities 'West Virginia' and 'Villanova'. However, no specific relations between these entities or other mentioned entities ('Frank Young') are explicitly stated in the sentence. Therefore, the answer would be: None.",
   'predicted_relations': 'None',
   'precision': 0.0,
   'recall': 0.0,
   'f1': 0}},
 {'input': "9:30 P.M. (13) THE WILD PARROTS OF TELEGRAPH HILL -- '' Independent Lens '' follows Mark Bittner , a homeless street musician in San Francisco , as he l

In [ ]:
import json
# Define the file path
file_path = "./mistral-finetuning-data.json"

# Save the data to a JSON file
with open(file_path, "w") as json_file:
    json.dump(data, json_file)

print(f"Data saved to {file_path}")


Data saved to ./mistral-finetuning-data.json


In [ ]:
# Calculate average metrics
avg_precision, avg_recall, avg_f1 = calculate_average_metrics(data)

print("Average Precision:", avg_precision)
print("Average Recall:", avg_recall)
print("Average F1-score:", avg_f1)


KeyError: 'base_model'

## load finetuned model

In [ ]:
! ls drive/MyDrive/mistral-finetune/runs

checkpoint-500			       mistral-finetuning-data.json
data				       mistral-finetuning-data-loweralpha.json
helper_functions.py		       mistral-finetuning-data-nonegative.json
mistral-finetuning-data2.json	       __pycache__
mistral-finetuning-data3.json	       runs
mistral-finetuning-data-balanced.json


In [16]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "mistralai/Mistral-7B-Instruct-v0.2"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,  # Mistral, same as before
    quantization_config=bnb_config,  # Same quantization config as before
    device_map="auto",
    trust_remote_code=True,
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

ImportError: Using `low_cpu_mem_usage=True` or a `device_map` requires Accelerate: `pip install accelerate`

In [ ]:
eval_tokenizer = AutoTokenizer.from_pretrained(model_id, add_bos_token=True, trust_remote_code=True)


In [19]:
! ls drive/MyDrive/mistral-finetune/runs/mistral7b-re-finetune/checkpoint-375


checkpoint-100	checkpoint-175	checkpoint-25	checkpoint-300	checkpoint-375
checkpoint-125	checkpoint-200	checkpoint-250	checkpoint-325	checkpoint-50
checkpoint-150	checkpoint-225	checkpoint-275	checkpoint-350	checkpoint-75


In [20]:
from peft import PeftModel

ft_model = PeftModel.from_pretrained(model, "drive/MyDrive/mistral-finetune/runs/mistral7b-re-finetune/checkpoint-375")


In [21]:
import json
with open("drive/MyDrive/mistral-finetune/mistral-finetuning-data.json", "r") as json_file:
    data = json.load(json_file)

In [22]:
data

[{'input': '-LRB- AP -RRB- WEST VIRGINIA 67 , VILLANOVA 56 -- Frank Young scored a career-high 25 points , including eight 3-pointers , for No. 25 West Virginia -LRB- 12-1 , 2-0 -RRB- in a Big East game .',
  'output': "('Frank Young', '/people/person/place_lived', 'West Virginia')",
  'base_model': {'response': " Based on the given sentence, there do not seem to be any explicit relations mentioned that can be directly extracted using the provided list. The sentence primarily describes an event (a basketball game) involving entities 'West Virginia' and 'Villanova'. However, no specific relations between these entities or other mentioned entities ('Frank Young') are explicitly stated in the sentence. Therefore, the answer would be: None.",
   'predicted_relations': 'None',
   'precision': 0.0,
   'recall': 0.0,
   'f1': 0}},
 {'input': "9:30 P.M. (13) THE WILD PARROTS OF TELEGRAPH HILL -- '' Independent Lens '' follows Mark Bittner , a homeless street musician in San Francisco , as he l

In [23]:
import sys
sys.path.append('drive/My Drive/mistral-finetune/')

In [89]:
import importlib

# Suppose your module is named 'mymodule'
import helper_functions

# Now, let's say you made some changes to mymodule.py
# You can reload the module to see the effects of changes
importlib.reload(helper_functions)

<module 'helper_functions' from '/content/drive/My Drive/mistral-finetune/helper_functions.py'>

In [90]:
from helper_functions import eval_formatting_func, get_tuples, get_relations, get_scores

In [92]:
get_scores(data[0]['output'], data[0]['ft_model']['predicted_relations'])

(0.5, 1.0, 0.6666666666666666)

In [93]:
from tqdm import tqdm

progress_bar = tqdm(total=len(data), desc='Progress')

for idx, example in zip(list(range(len(data))), data):

    eval_prompt = eval_formatting_func(example)

    model_input = eval_tokenizer(eval_prompt, return_tensors="pt").to("cuda")

    ft_model.eval()
    with torch.no_grad():
        response  = eval_tokenizer.decode(ft_model.generate(**model_input, max_new_tokens=256,
                                                         repetition_penalty=1.15,
                                                         pad_token_id=eval_tokenizer.eos_token_id)[0],
                                          skip_special_tokens=True)
        response = response.replace(eval_prompt, "")

    predicted_relations = get_tuples(response)

    expected_relations = get_relations(example)

    precision, recall,f1 = get_scores(expected_relations, predicted_relations)

    data[idx].update({'ft_model':
                        {'response': response,
                         'predicted_relations':predicted_relations,
                        'precision': precision,
                        'recall': recall,
                        'f1': f1}
                    })
    progress_bar.update(1)

progress_bar.close()

Progress:   0%|          | 0/100 [00:00<?, ?it/s]

TypeError: eval() arg 1 must be a string, bytes or code object

In [106]:
data[0]['ft_model']

{'response': "\n('Frank Young', '/people/person/place_of_birth', 'West Virginia'), ('Frank Young', '/people/person/place_lived', 'West Virginia')",
 'predicted_relations': [('Frank Young',
   '/people/person/place_of_birth',
   'West Virginia'),
  ('Frank Young', '/people/person/place_lived', 'West Virginia')],
 'precision': 0.5,
 'recall': 1.0,
 'f1': 1.0}

In [109]:
for ixd in range(len(data)):
    p,r,f= get_scores(data[idx]['output'], data[idx]['base_model']['predicted_relations'])
    data[idx]['base_model']['precision'] =p
    data[idx]['base_model']['recall'] = r
    data[idx]['base_model']['f1'] =r

In [105]:
for ixd in range(len(data)):
    p,r,f= get_scores(data[idx]['output'], data[idx]['ft_model']['predicted_relations'])
    data[idx]['ft_model']['precision'] =p
    data[idx]['ft_model']['recall'] = r
    data[idx]['ft_model']['f1'] =r

In [27]:
# Define the file path
file_path = "drive/MyDrive/mistral-finetune/mistral-finetuning-data-r16alpha32.json"

# Save the data to a JSON file
with open(file_path, "w") as json_file:
    json.dump(data, json_file)

print(f"Data saved to {file_path}")


Data saved to drive/MyDrive/mistral-finetune/mistral-finetuning-data-r16alpha32.json


In [ ]:
"""# Define the file path
import json
file_path = "drive/MyDrive/mistral-finetune/mistral-finetuning-data-balanced.json"

# Save the data to a JSON file
with open(file_path, "r") as json_file:
    data = json.load(json_file)"""


In [111]:
def calculate_average_metrics2(data):
    total_precision = 0.0
    total_recall = 0.0
    total_f1 = 0.0
    total_samples = len(data)

    for item in data:
        total_precision += item['base_model']['precision']
        total_recall += item['base_model']['recall']
        total_f1 += item['base_model']['f1']

    average_precision = total_precision / total_samples
    average_recall = total_recall / total_samples
    average_f1 = total_f1 / total_samples

    return average_precision, average_recall, average_f1

# Calculate average metrics
avg_precision, avg_recall, avg_f1 = calculate_average_metrics2(data)

print("Average Precision:", avg_precision)
print("Average Recall:", avg_recall)
print("Average F1-score:", avg_f1)

Average Precision: 0.10166666666666666
Average Recall: 0.14
Average F1-score: 0.10796825396825399


In [110]:
def calculate_average_metrics2(data):
    total_precision = 0.0
    total_recall = 0.0
    total_f1 = 0.0
    total_samples = len(data)

    for item in data:
        total_precision += item['ft_model']['precision']
        total_recall += item['ft_model']['recall']
        total_f1 += item['ft_model']['f1']

    average_precision = total_precision / total_samples
    average_recall = total_recall / total_samples
    average_f1 = total_f1 / total_samples

    return average_precision, average_recall, average_f1

# Calculate average metrics
avg_precision, avg_recall, avg_f1 = calculate_average_metrics2(data)

print("Average Precision:", avg_precision)
print("Average Recall:", avg_recall)
print("Average F1-score:", avg_f1)


Average Precision: 0.13
Average Recall: 0.09833333333333333
Average F1-score: 0.10952380952380952


https://huggingface.co/docs/transformers/en/llm_tutorial

### common pitfalls

Wrong padding side

LLMs are decoder-only architectures, meaning they continue to iterate on your input prompt. If your inputs do not have the same length, they need to be padded. Since LLMs are not trained to continue from pad tokens, your input needs to be left-padded. Make sure you also don’t forget to pass the attention mask to generate!




In [88]:
for x in data:
    if "none" in x['ft_model']['response'].lower():
        print(("Correct: ", x['output'], "Predicted: ", x['ft_model']['response']))
        print("")

In [84]:
for x in data:
    if "None" in x['output']:
        print(("Correct: ", x['output'], "Predicted: ", x['ft_model']['response']))
        print("")

('Correct: ', 'None', 'Predicted: ', "\n('Paramus', '/location/located_in', 'New Jersey')")

('Correct: ', 'None', 'Predicted: ', "\n('Brian Dawkins', '/people/person/place_lived', 'Philadelphia')")

('Correct: ', 'None', 'Predicted: ', "\n('Ochoa', '/people/person/place_of_birth', 'Mexico'), ('Ochoa', '/people/person/place_lived', 'Mexico')")

('Correct: ', 'None', 'Predicted: ', "\n('Terrence Malick', '/people/person/place_lived', 'Saskatchewan')")

('Correct: ', 'None', 'Predicted: ', "\n('Gazprom', '/business/location', 'Russia')")

('Correct: ', 'None', 'Predicted: ', "\n('Yellowstone Club', '/location/located_in', 'Montana')")

('Correct: ', 'None', 'Predicted: ', "\n('Khartoum', '/location/administrative_division/country', 'Sudan'), ('Khartoum', '/location/located_in', 'Sudan')")

('Correct: ', 'None', 'Predicted: ', "\n('San Francisco Bay Area', '/location/located_in', 'Boston')")

('Correct: ', 'None', 'Predicted: ', "\n('Fort Bend County', '/location/located_in', 'Texas')")



In [70]:
idx=10
expected=[eval(data[idx]['output'])]
expected

[None]

In [71]:
predicted = data[idx]['ft_model']['predicted_relations']
predicted

[('Ochoa', '/people/person/place_of_birth', 'Mexico'),
 ('Ochoa', '/people/person/place_lived', 'Mexico')]

In [72]:
expected_set = set(expected)
expected_set


{None}

In [73]:
predicted_set = set(predicted)
predicted_set

{('Ochoa', '/people/person/place_lived', 'Mexico'),
 ('Ochoa', '/people/person/place_of_birth', 'Mexico')}

In [74]:
true_positives = len(expected_set.intersection(predicted_set))
true_positives

0

In [75]:
# share of actual relationships that we identified
recall = true_positives / len(expected_set) if len(expected_set) > 0 else 0
recall

0.0

In [76]:
# share of predicted relationships that are actually true
precision = true_positives / len(predicted_set) if len(predicted_set) > 0 else 0
precision

0.0

In [ ]:
    precision = true_positives / len(predicted_set) if len(predicted_set) > 0 else 0


In [ ]:
precision = true_positives / len(predicted_set) if len(predicted_set) > 0 else 0


In [38]:
get_scores(data[0]['output'], data[0]['ft_model']['predicted_relations'])

(0.0, 0.0, 0)

In [31]:

[ x['ft_model']['response'] for x in data]

["\n('Frank Young', '/people/person/place_of_birth', 'West Virginia'), ('Frank Young', '/people/person/place_lived', 'West Virginia')",
 "\n('Mark Bittner', '/people/person/place_lived', 'San Francisco')",
 "\n('Mstislav Rostropovich', '/people/deceasedperson/place_of_death', 'Paris')",
 "\n('Paramus', '/location/located_in', 'New Jersey')",
 "\n('Brian Dawkins', '/people/person/place_lived', 'Philadelphia')",
 "\n('Guantánamo Bay', '/location/located_in', 'Cuba')",
 "\n('Peter A. Chernin', '/business/person/company', 'News Corporation')\n('News Corporation', '/business/company/advisors', 'Peter A. Chernin'), ('Peter A. Chernin', '/business/company/advisors', 'News Corporation')\n('Ron Meyer', '/business/person/company', 'Universal Studios')\n('Universal Studios', '/business/company/advisors', 'Ron Meyer'), ('Ron Meyer', '/business/person/company', 'Universal Studios')\n('Richard Cook', '/business/person/company', 'Walt Disney Studios')\n('Walt Disney Studios', '/business/company/advis

https://stats.stackexchange.com/questions/164876/what-is-the-trade-off-between-batch-size-and-number-of-iterations-to-train-a-neu